In [1]:
import re
import numpy as np


In [2]:
input = '''498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
'''

# with open('input') as f:
#     input = f.read()


In [3]:
coordinates = re.findall('(\d+),(\d+)', input)
x_start = min([int(x[0]) for x in coordinates]) - 201
x_end = max([int(x[0]) for x in coordinates]) + 202
y_start = 0  # source of the sand
y_end = max([int(x[1]) for x in coordinates]) + 3
pos_sand = (500,0)


In [4]:
def tc(coordinate):
    # translate coordinate (x,y) to (m,n)
    return coordinate[1] - y_start, coordinate[0] - x_start


In [5]:
# create cave
shape = (y_end - y_start, x_end - x_start)
cave = np.full(shape, '.')


In [6]:
# insert rocks
for rock in input.splitlines():
    coordinates = re.findall('(\d+),(\d+)', rock)
    coordinates = [(int(x[0]), int(x[1])) for x in coordinates]
    currPoint = coordinates[0]
    cave[tc(currPoint)] = '#'
    for nextPoint in coordinates[1:]:
        while currPoint != nextPoint:
            step = (np.sign(nextPoint[0] - currPoint[0]), np.sign(nextPoint[1] - currPoint[1]))
            currPoint = (currPoint[0] + step[0], currPoint[1] + step[1])
            cave[tc(currPoint)] = '#'

# insert source of the sand
cave[tc(pos_sand)] = '+'


In [7]:
OUT_OF_CAVE = (-1, -1)


def move_sand(position):
    # check if sand falls into the abyss
    y_next = position[1] + 1
    if y_next >= y_end:
        return OUT_OF_CAVE
    
    # calc next position of the sand
    nextPos = [(position[0], y_next), (position[0] - 1, y_next), (position[0] + 1, y_next)]
        
    # move down
    if cave[tc(nextPos[0])] == '.':
        return nextPos[0]
    
    # move down and left
    if cave[tc(nextPos[1])] == '.':
        return nextPos[1]
    
    # move down and right
    if cave[tc(nextPos[2])] == '.':
        return nextPos[2]
    
    # sand stays in the same position
    return position


def find_rest():
    # calc rest position of the sand
    pos_nextSand_prev = pos_sand
    
    while True:
        pos_nextSand = move_sand(pos_nextSand_prev)
        # sand falls into the abyss
        if pos_nextSand == OUT_OF_CAVE:
            return OUT_OF_CAVE
        # sand comes to rest (no more movement possible)
        if pos_nextSand == pos_nextSand_prev:
            return pos_nextSand
        
        pos_nextSand_prev = pos_nextSand


In [8]:
# fill cave with sand
sandUnits = 0
while True:
    pos_nextSand = find_rest()
    if pos_nextSand == OUT_OF_CAVE:
        break
    cave[tc(pos_nextSand)] = 'o'
    sandUnits += 1


In [9]:
# np.savetxt('cave.txt', cave, fmt='%c')


In [10]:
print(sandUnits)


24
